In [1]:
#imports
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import os

import torch
import torchvision
from torchvision import ops
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
#To use in Colab
!git clone https://github.com/gabriellecaillaud/APS360_Traffic_Sign_Recognition.git

fatal: destination path 'APS360_Traffic_Sign_Recognition' already exists and is not an empty directory.


In [3]:
#Load the custom modules on colab based on the files on git
import sys
 #To use on colab
sys.path.append('/content/APS360_Traffic_Sign_Recognition')

# might be neccessary to change to from RCNN_model.utils import * in model
from RCNN_model.model import *
from RCNN_model.utils import *

In [4]:
#path csv with labels
csv_path = "/content/APS360_Traffic_Sign_Recognition/dataset_traffic_signs.csv"

In [12]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the csv file
df = pd.read_csv(csv_path)

# Split the data into train and test sets
train_val_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Split the train_val_data into train and validation sets
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=42)

# Save the train, validation, and test sets to csv files

train_data.to_csv('train.csv', index=False)
val_data.to_csv('val.csv', index=False)
test_data.to_csv('test.csv', index=False)


In [13]:
#The goal of this cell is to split the train.csv into five .csv files
# to avoid the memory to be full

import pandas as pd

# Read the input CSV file into a pandas DataFrame
df = pd.read_csv('train.csv')

# Calculate the number of rows per output file
rows_per_file = len(df) // 5

# Split the DataFrame into five smaller DataFrames
dfs = [df[i:i+rows_per_file] for i in range(0, len(df), rows_per_file)]

# Write each smaller DataFrame to a CSV file
for i, df in enumerate(dfs):
    df.to_csv(f'train_part_{i+1}.csv', index=False)


In [17]:
df = pd.read_csv('train_part_1.csv')

In [18]:
df

,Id,imageUrl,annotation.0.centerX,annotation.0.centerY,annotation.0.width,annotation.0.height,annotation.0.classification,annotation.1.centerX,annotation.1.centerY,annotation.1.width,annotation.1.height,annotation.1.classification
0,4063,AugDataNoRight/img350.png,0.312339,0.444087,0.089974,0.106684,noRight,NaN,NaN,NaN,NaN,NaN
1,2336,AugData/NoEntry/img4377.png,0.215084,0.541589,0.098696,0.124767,NoEntry,NaN,NaN,NaN,NaN,NaN
2,271,AugData/100kmh/img3084.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3161,AugData/Stop/img427.png,0.692130,0.351235,0.186728,0.194444,Stop,NaN,NaN,NaN,NaN,NaN
4,1026,AugData/yield/img1532.png,0.574550,0.333548,0.185090,0.176093,yield,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
547,98,AugData/100kmh/img1569.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,503,AugData/100kmh/img545.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,4180,AugDataNoRight/img4470.png,0.307841,0.142674,0.170951,0.192802,noRight,NaN,NaN,NaN,NaN,NaN
550,116,AugData/100kmh/img1587.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#imports
from tqdm import tqdm
import pickle

In [5]:
class ObjectDetectionDataset(Dataset):
    '''
    A Pytorch Dataset class to load the images and their corresponding annotations.
    
    Returns
    ------------
    images: torch.Tensor of size (B, C, H, W)
    gt bboxes: torch.Tensor of size (B, max_objects, 4)
    gt classes: torch.Tensor of size (B, max_objects)
    '''
    def __init__(self, csv_path, img_size, name2idx):
        self.annotation_path = csv_path
        self.img_size = img_size
        self.name2idx = name2idx
        
        self.img_data_all, self.gt_bboxes_all, self.gt_classes_all = self.get_data()
        
    def __len__(self):
        return self.img_data_all.size(dim=0)
    
    def __getitem__(self, idx):
        return self.img_data_all[idx], self.gt_bboxes_all[idx], self.gt_classes_all[idx]
        
    def get_data(self):
        img_data_all = []
        gt_idxs_all = []
        
        gt_boxes_all, gt_classes_all, img_paths = parse_annotation(self.annotation_path, self.img_size)
        
        for i, img_path in tqdm(enumerate(img_paths), total=len(img_paths)):
      
            # skip if the image path is not valid
            if (not img_path) or (not os.path.exists(img_path)):
                continue
            
            # read and resize image
            
            img = io.imread(img_path)
            img = resize(img, self.img_size)
            
            # convert image to torch tensor and reshape it so channels come first
            img_tensor = torch.from_numpy(img).permute(2, 0, 1)
            
            # encode class names as integers
            gt_classes = gt_classes_all[i]
            gt_idx = torch.Tensor([self.name2idx[name] for name in gt_classes])
            
            img_data_all.append(img_tensor)
            gt_idxs_all.append(gt_idx)
        
        # pad bounding boxes and classes so they are of the same size

        if len(gt_boxes_all)!=0 and len(gt_idxs_all)!=0 :
          
          gt_bboxes_pad = pad_sequence(gt_boxes_all, batch_first=True, padding_value=-1)
          gt_classes_pad = pad_sequence(gt_idxs_all, batch_first=True, padding_value=-1)
        
        # stack all images
        img_data_stacked = torch.stack(img_data_all)[:, :3, :, :]
        
        return img_data_stacked.to(dtype=torch.float32), gt_bboxes_pad, gt_classes_pad

    def __getstate__(self):
        state = self.__dict__.copy()
        state['img_data_all'] = pickle.dumps(state['img_data_all'])
        state['gt_bboxes_all'] = pickle.dumps(state['gt_bboxes_all'])
        state['gt_classes_all'] = pickle.dumps(state['gt_classes_all'])
        return state

    def __setstate__(self, state):
        state['img_data_all'] = pickle.loads(state['img_data_all'])
        state['gt_bboxes_all'] = pickle.loads(state['gt_bboxes_all'])
        state['gt_classes_all'] = pickle.loads(state['gt_classes_all'])
        self.__dict__.update(state)

In [6]:
img_width = 640
img_height = 480
csv_path = "/content/APS360_Traffic_Sign_Recognition/dataset_traffic_signs.csv"
image_dir = os.path.join("data", "images")
name2idx = {'pad': -1, '30kmh': 0,'60kmh':1, '100kmh' : 2, 'yield': 3, 'keepRight' :4, 'NoEntry':5, 'NoLeft': 6, 'Stop':7, 'noRight':8, 'ChildrenCrossing' :9 }
idx2name = {v:k for k, v in name2idx.items()}

In [7]:
#Creating a Pytorch Dataset file using custom class

od_dataset_train_1 = ObjectDetectionDataset("train_part_1.csv", (img_height, img_width), name2idx)

100%|██████████| 463/463 [00:39<00:00, 11.61it/s]


In [ ]:
#trying to save the dataset. It won't work on Colab, the memory will be full
torch.save(od_dataset_train_1, 'od_dataset_train_1.pt')

In [9]:
#Creating a second subdataset
od_dataset_train_2 = ObjectDetectionDataset("train_part_2.csv", (img_height, img_width), name2idx)

100%|██████████| 436/436 [00:35<00:00, 12.19it/s]


In [10]:
#concatenating the two  small datasets
train_dev_sets = torch.utils.data.ConcatDataset([od_dataset_train_1, od_dataset_train_2])
#train_dev_loader = DataLoader(dataset=train_dev_sets, ...)

In [10]:
#useful for the model definition
out_c, out_h, out_w = 2048 ,15 ,20


In [8]:
#Dataloading
train_data1_loader = DataLoader(od_dataset_train_1, batch_size=16)

In [12]:
img_size = (img_height, img_width)
out_size = (out_h, out_w) ## see other d
n_classes = len(name2idx) - 1 # exclude pad idx
roi_size = (2, 2)

#the model
detector = TwoStageDetector(img_size, out_size, out_c, n_classes, roi_size)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 318MB/s]


In [ ]:
import torch.optim as optim

In [13]:
def training_loop(model, learning_rate, train_dataloader, n_epochs):
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model.train()
    loss_list = []
    
    for i in tqdm(range(n_epochs)):
        total_loss = 0
        for img_batch, gt_bboxes_batch, gt_classes_batch in train_dataloader:
            
            # forward pass
            loss = model(img_batch, gt_bboxes_batch, gt_classes_batch)
            
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        loss_list.append(total_loss)
        
    return loss_list

In [ ]:
learning_rate = 1e-1
n_epochs = 2

loss_list = training_loop(detector, learning_rate, train_data1_loader, n_epochs)

 50%|█████     | 1/2 [21:22<21:22, 1282.36s/it]